In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import datetime, timedelta
import requests
import openpyxl
from pandas_market_calendars import get_calendar
from requests.adapters import HTTPAdapter
from requests.exceptions import RequestException, HTTPError, ConnectionError, Timeout

# Funções

In [2]:
def workday() -> datetime:
    return get_calendar('BMF').schedule(
        data_ini=(datetime.now() - timedelta(days=90)).strftime("%d/%m/%Y"),
        data_fim=datetime.now().strftime("%d/%m/%Y")
    ).loc[::-1, ("market_open")].iloc[0].date()

# Main

In [3]:
USER = Path(os.path.expanduser("~"))

In [4]:
excel_path = USER / r"C:\Users\GiordanoBrunoGava\OneDrive - WISIR\General - WISIR\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx"
dados_cotacao__aj_consolidado_excel = pd.read_excel(excel_path)

In [5]:
dados_cotacao__aj_consolidado_excel

,DATA,IBOV,SMAL11,IDIV,BDRX,PATI3,PATI4,BAHI3,BBDC3,BBDC4,...,NASD11,USSX34,XINA11,JURO11,KORE11,CPTI11,BODB11,BDIF11,XPID11,OGIN11
0,2023-03-06,104700,"89,9",7052,11769,NaN,NaN,"10,804458677","10,9511139398","12,4342542203",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-03-07,104227,"90,5",7018,11690,NaN,NaN,"10,9343920376","11,1099611556","12,7242932578",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-03-08,106540,"93,25",7127,11594,NaN,NaN,"10,6045611992","11,3996237257","13,1172493731",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-03-09,105071,"92,8",7017,11424,NaN,NaN,"10,7644791814","11,3435600025","13,0704688832",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-03-10,103618,"91,38",6934,11362,NaN,NaN,"10,7644791814","11,0445534785","12,6962249638",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2024-06-17,119137,93,8400,19171,26,NaN,NaN,"11,0627356954","12,5743343907",...,"15,06",NaN,"5,96","106,73","103,55","97,6","9,28","87,79","67,3","9,7"
284,2024-06-18,119630,"93,3",8463,19264,NaN,NaN,"6,45","10,98","12,38",...,"15,1",NaN,"5,98","106,99","104,69","97,69","9,3","88,19","65,4","9,8"
285,2024-06-19,120261,"93,49",8510,19432,NaN,NaN,NaN,"11,04","12,44",...,"15,23",NaN,"5,94","106,71","104,92","97,7","9,3","88,15",66,"9,76"
286,2024-06-20,120445,94,8560,19254,NaN,NaN,"6,7","11,07","12,41",...,"15,05","203,16","6,02","106,85","104,77","97,36","9,38","88,69","66,3","9,74"


In [6]:
dados_cotacao__aj_consolidado_excel['DATA'] = pd.to_datetime(dados_cotacao__aj_consolidado_excel['DATA'], format="%d/%m/%Y")

In [7]:
ultima_linha_com_data = dados_cotacao__aj_consolidado_excel['DATA'].last_valid_index()

In [8]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini=(datetime.now() - timedelta(days=60)).strftime("%d/%m/%Y")
data_fim=datetime.now().strftime("%d/%m/%Y") 

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini}%26data_fim%3D{data_fim}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}
        
#SOLICITAÇÃO DA API
response = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [9]:
novos_dados = response.json()

In [10]:
data = list()
for lin in (table := response.json()["tables"]["tab0"]):
    data.append(table[lin])

In [11]:
dados_cotacao__aj_novos = pd.DataFrame(data)

In [12]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.rename(
    columns = {
        # Dict comprehension para colocar os nomes antigos da coluna com os valores da primeira linha.
        nome_antigo_da_coluna: nome_novo_da_coluna for
            nome_antigo_da_coluna, nome_novo_da_coluna in
                # Zip para poder fazer unpack das duas informacoes
                zip(
                    list(dados_cotacao__aj_novos.columns), [
                        # Como os dados da primeira linha vem em formatod indeseja ja formatamos aqui
                        str(item).replace("\nPRECO_AJ(,,,A,C)", "") for
                            item in
                                # iloc para pegar a primeira linha e todas as colunas
                                list(dados_cotacao__aj_novos.iloc[0, :])
                    ]
                )
        }
)

In [13]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(index = 0)

In [14]:
# ARRUMA AS DATAS ERRADAS PARA O FORMATO DAS OUTRAS
parser.parse("12-06-2024", dayfirst = True)

datetime.datetime(2024, 6, 12, 0, 0)

In [15]:
dados_cotacao__aj_consolidado_excel

,DATA,IBOV,SMAL11,IDIV,BDRX,PATI3,PATI4,BAHI3,BBDC3,BBDC4,...,NASD11,USSX34,XINA11,JURO11,KORE11,CPTI11,BODB11,BDIF11,XPID11,OGIN11
0,2023-03-06,104700,"89,9",7052,11769,NaN,NaN,"10,804458677","10,9511139398","12,4342542203",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-03-07,104227,"90,5",7018,11690,NaN,NaN,"10,9343920376","11,1099611556","12,7242932578",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-03-08,106540,"93,25",7127,11594,NaN,NaN,"10,6045611992","11,3996237257","13,1172493731",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-03-09,105071,"92,8",7017,11424,NaN,NaN,"10,7644791814","11,3435600025","13,0704688832",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-03-10,103618,"91,38",6934,11362,NaN,NaN,"10,7644791814","11,0445534785","12,6962249638",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2024-06-17,119137,93,8400,19171,26,NaN,NaN,"11,0627356954","12,5743343907",...,"15,06",NaN,"5,96","106,73","103,55","97,6","9,28","87,79","67,3","9,7"
284,2024-06-18,119630,"93,3",8463,19264,NaN,NaN,"6,45","10,98","12,38",...,"15,1",NaN,"5,98","106,99","104,69","97,69","9,3","88,19","65,4","9,8"
285,2024-06-19,120261,"93,49",8510,19432,NaN,NaN,NaN,"11,04","12,44",...,"15,23",NaN,"5,94","106,71","104,92","97,7","9,3","88,15",66,"9,76"
286,2024-06-20,120445,94,8560,19254,NaN,NaN,"6,7","11,07","12,41",...,"15,05","203,16","6,02","106,85","104,77","97,36","9,38","88,69","66,3","9,74"


In [16]:
dados_cotacao__aj_novos["Data"] = dados_cotacao__aj_novos["Data"].map(lambda row: parser.parse(row, dayfirst = True))

In [17]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.rename(columns = {"Data": "DATA"}).astype(dtype = {"DATA": "datetime64[ns]"})

In [18]:
dados_cotacao__aj_novos.reset_index(drop = True)

,DATA,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,BHIA3,INBR32,MELI34,JURO11,KORE11,CPTI11,BODB11,BDIF11,XPID11,OGIN11
0,2024-04-25,"290,51","120,91","12,54","188,260381216","5,42",124645,"99,4",8578,16264,...,"5,52","26,15",59,"104,188770327","105,842026306","99,6455691425","9,28021276596","88,1582045832","71,7357635332","9,61131368027"
1,2024-04-26,"289,8","122,83","12,62",,"5,46",126526,"101,76",8673,16200,...,"5,44","26,6","60,5","104,169224549","105,441405638","99,1069444444","9,30989361702","88,4515763122","71,6171756888","9,56262414794"
2,2024-04-29,"290,95","123,77","12,67","188,080952705","5,52",127351,"102,85",8735,16257,...,"7,3","26,02","61,39","104,501502777","106,01790855","99,244048913","9,31978723404","89,1850056349","71,2416475149","9,54314833502"
3,2024-04-30,291,"122,38","12,61","190,024761576","5,48",125924,"100,5",8679,16293,...,"6,85",26,"63,67","104,423319665","106,946175953","99,1069444444","9,3","88,2070998713","71,38","9,65026530612"
4,2024-05-02,288,"123,35","12,5","187,991238449","5,73",127122,"101,8",8842,16141,...,"7,89","26,48","68,16","104,729865395","106,649652359","99,0079365079","9,33","88,29511139","70,2","9,65026530612"
5,2024-05-03,"289,55","124,95","12,64","185,689559752","5,72",128508,105,8962,16263,...,"7,95","27,71","68,76","104,729865395","107,03513303","99,8594047619","9,38","88,6667155802","70,51","9,67"
6,2024-05-06,"292,1","124,71","12,8","191,286886227","5,72",128465,"103,1",8943,16449,...,"6,97","27,62","69,68","104,710088251","106,451969964","99,7009920635","9,33","89,7619700354","72,94","9,62066326531"
7,2024-05-07,"292,25","125,4","12,79","193,412074354","5,66",129210,"103,6",8914,16421,...,"7,03","27,72","71,59","104,621091104","107,015364791","99,5524801587","9,32","88,9894244822","73,05","9,61079591837"
8,2024-05-08,"293,4","125,75","12,82","195,178075755","5,63",129480,"104,07",8937,16481,...,"6,96","27,74","72,6","105,105631129","107,736905534","99,403968254","9,35","88,5395878309","71,99","9,61079591837"
9,2024-05-09,"298,3","124,52","12,99",,"5,83",128188,"101,75",8837,16680,...,"7,12","29,7","72,4","105,184739704","107,341540743","99,0772420635","9,33","88,0115187186","71,04","9,61079591837"


In [19]:
dados_cotacao__aj_novos

,DATA,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,BHIA3,INBR32,MELI34,JURO11,KORE11,CPTI11,BODB11,BDIF11,XPID11,OGIN11
1,2024-04-25,"290,51","120,91","12,54","188,260381216","5,42",124645,"99,4",8578,16264,...,"5,52","26,15",59,"104,188770327","105,842026306","99,6455691425","9,28021276596","88,1582045832","71,7357635332","9,61131368027"
2,2024-04-26,"289,8","122,83","12,62",,"5,46",126526,"101,76",8673,16200,...,"5,44","26,6","60,5","104,169224549","105,441405638","99,1069444444","9,30989361702","88,4515763122","71,6171756888","9,56262414794"
3,2024-04-29,"290,95","123,77","12,67","188,080952705","5,52",127351,"102,85",8735,16257,...,"7,3","26,02","61,39","104,501502777","106,01790855","99,244048913","9,31978723404","89,1850056349","71,2416475149","9,54314833502"
4,2024-04-30,291,"122,38","12,61","190,024761576","5,48",125924,"100,5",8679,16293,...,"6,85",26,"63,67","104,423319665","106,946175953","99,1069444444","9,3","88,2070998713","71,38","9,65026530612"
5,2024-05-02,288,"123,35","12,5","187,991238449","5,73",127122,"101,8",8842,16141,...,"7,89","26,48","68,16","104,729865395","106,649652359","99,0079365079","9,33","88,29511139","70,2","9,65026530612"
6,2024-05-03,"289,55","124,95","12,64","185,689559752","5,72",128508,105,8962,16263,...,"7,95","27,71","68,76","104,729865395","107,03513303","99,8594047619","9,38","88,6667155802","70,51","9,67"
7,2024-05-06,"292,1","124,71","12,8","191,286886227","5,72",128465,"103,1",8943,16449,...,"6,97","27,62","69,68","104,710088251","106,451969964","99,7009920635","9,33","89,7619700354","72,94","9,62066326531"
8,2024-05-07,"292,25","125,4","12,79","193,412074354","5,66",129210,"103,6",8914,16421,...,"7,03","27,72","71,59","104,621091104","107,015364791","99,5524801587","9,32","88,9894244822","73,05","9,61079591837"
9,2024-05-08,"293,4","125,75","12,82","195,178075755","5,63",129480,"104,07",8937,16481,...,"6,96","27,74","72,6","105,105631129","107,736905534","99,403968254","9,35","88,5395878309","71,99","9,61079591837"
10,2024-05-09,"298,3","124,52","12,99",,"5,83",128188,"101,75",8837,16680,...,"7,12","29,7","72,4","105,184739704","107,341540743","99,0772420635","9,33","88,0115187186","71,04","9,61079591837"


In [20]:
if dados_cotacao__aj_consolidado_excel.index.duplicated().any():
    dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.reset_index(drop=True)
if dados_cotacao__aj_novos.index.duplicated().any():
    dados_cotacao__aj_novos = dados_cotacao__aj_novos.reset_index(drop=True)

In [21]:
dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.loc[:, ~dados_cotacao__aj_consolidado_excel.columns.duplicated()]
dados_cotacao__aj_novos = dados_cotacao__aj_novos.loc[:, ~dados_cotacao__aj_novos.columns.duplicated()]

In [24]:
# SALVA O DF CONCATENANDO NO EXCEL
output_path = USER / r'OneDrive - WISIR\General - WISIR\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx'

dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(dados_cotacao__aj_novos.index[-1])
dados_cotacao__aj_novos.to_excel(output_path, index=False)

df = pd.read_excel(output_path)
df.replace(['-', '[]'], np.nan, inplace=True)
df = df.dropna(how='all', subset=df.columns.difference(['DATA']))
df.to_excel(output_path, index=False)


print('Dados atualizados e salvos no Excel.')
print('URL construída:', url + payload)

Dados atualizados e salvos no Excel.
URL construída: https://www.comdinheiro.com.br/Clientes/API/EndPoint001.phpusername=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D25/04/2024%26data_fim%3D24/06/2024%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3
